In [4]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Amazon_Instant_Video_5.json.gz

--2017-04-07 22:47:18--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Amazon_Instant_Video_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9517526 (9.1M) [application/x-gzip]
Saving to: ‘reviews_Amazon_Instant_Video_5.json.gz’

reviews_Amazon_Inst 100%[===================>]   9.08M  3.94MB/s    in 2.3s    

2017-04-07 22:47:21 (3.94 MB/s) - ‘reviews_Amazon_Instant_Video_5.json.gz’ saved [9517526/9517526]



In [7]:
import json

In [13]:
with open("reviews_Amazon_Instant_Video_5.json") as f:
    data = []
    for line in f:
        data.append(json.loads(line))

In [14]:
len(data)

37126

In [2]:
import pandas as pd

In [16]:
moviedf =pd.DataFrame(data)

In [18]:
moviedf.head(5)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,B000H00VBQ,"[0, 0]",2.0,I had big expectations because I love English ...,"05 3, 2014",A11N155CW1UV02,AdrianaM,A little bit boring for me,1399075200
1,B000H00VBQ,"[0, 0]",5.0,I highly recommend this series. It is a must f...,"09 3, 2012",A3BC8O2KCL29V2,Carol T,Excellent Grown Up TV,1346630400
2,B000H00VBQ,"[0, 1]",1.0,This one is a real snoozer. Don't believe anyt...,"10 16, 2013",A60D5HQFOTSOM,"Daniel Cooper ""dancoopermedia""",Way too boring for me,1381881600
3,B000H00VBQ,"[0, 0]",4.0,Mysteries are interesting. The tension betwee...,"10 30, 2013",A1RJPIGRSNX4PW,"J. Kaplan ""JJ""",Robson Green is mesmerizing,1383091200
4,B000H00VBQ,"[1, 1]",5.0,"This show always is excellent, as far as briti...","02 11, 2009",A16XRPF40679KG,Michael Dobey,Robson green and great writing,1234310400


In [20]:
moviedf["asin"].value_counts()[0:5]

B00I3MPDP4    455
B00DAHSVYC    444
B00I3MMN4I    417
B00APE00H4    384
B00CDBTQCW    358
Name: asin, dtype: int64

In [35]:
movie1 = moviedf[moviedf["asin"] == "B00I3MPDP4"]

In [43]:
movie1_good = movie1[movie1["overall"]>3]
movie1_bad = movie1[movie1["overall"]<4]

In [22]:
import collections
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint

In [127]:
def word_tokenizer(text):
    #tokenizes and stems the text
    text = text.strip(",;'")
    tokens = word_tokenize(text.lower())
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(t) for t in tokens if t not in stopwords.words('english')]
    return tokens

In [128]:
def cluster_sentences(sentences, nb_of_clusters=5):
    tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenizer,
                                    stop_words=stopwords.words('english'),
                                    max_df=0.9,
                                    min_df=0.1,
                                    lowercase=True)
    #builds a tf-idf matrix for the sentences
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    kmeans = KMeans(n_clusters=nb_of_clusters)
    kmeans.fit(tfidf_matrix)
    clusters = collections.defaultdict(list)
    for i, label in enumerate(kmeans.labels_):
            clusters[label].append(i)
    return dict(clusters)

In [129]:
sentences_good = list(movie1_good["reviewText"])
sentences_bad = list(movie1_bad["reviewText"])

In [130]:
nclusters= 5

In [131]:
clusters_good = cluster_sentences(sentences_good, nclusters)
clusters_bad = cluster_sentences(sentences_bad, nclusters)

In [132]:
def print_clusters(cluster,sentences):
    for cluster in range(nclusters):
        print("cluster ",cluster,":")
        for i,sentence in enumerate(clusters[cluster]):
            print("\tsentence ",i,": ",sentences[sentence])

In [133]:
print_clusters(clusters_good, sentences_good)

cluster  0 :
	sentence  0 :  hopefully this one continues, great show. i don't like having to type a required amount of words for these reviews!
	sentence  1 :  I love a good murder mystery. Great writing. Well casted. Great acting.  It hooked me fairly quickly and I am ready for more.  Imho, this show was the best of the five pilots I watched in contention for the Amazon originals.  There was not much I didn't like.  I am buckled up and ready for the ride.
	sentence  2 :  I enjoyed the pilot episode and my interest is peaked enough to want to see where the show goes, I think it has a lot of potential and If they do it right it will be an excellent show
	sentence  3 :  Another great one from Amazon.This show has tremendous potential. First episode gives enough of a story to captivate and want to continue watching.Great acting, awesome story line, i look forward to seeing more of this show.
	sentence  4 :  This show is GREAT! Great writing, acting, filming, editing, casting and they kno

IndexError: list index out of range

In [134]:
print_clusters(clusters_bad, sentences_bad)

cluster  0 :
	sentence  0 :  Some of the acting to me is less than desirable.  It seems like some minor characters' roles are written to give their small parts too much significance.  However, I definitely want to know more.  The clues about Detective Bosch's past give you enough to speculate, but not quite enough to make the answers predictable.  A perfect mixture for great suspense.
	sentence  1 :  The first episode worked for me but not for my wife.  The courtroom drama is a distraction and the attorneys on both sides are real cliches.  I also weary of the rogue detective who has more conflict with his police peers than he does with the bad guys.  Fox's &#34;The Following&#34; is playing out the same worn out bit right now.  I'll try another episode, but it better get tighter and richer soon or I'm gone.
	sentence  2 :  Didn't find the storyline engaging.  Characters seemed to do the best they could.  Script is needs more effort to engage the viewer
	sentence  3 :  I thought this wa

IndexError: list index out of range

In [135]:
from sklearn.feature_extraction.text import CountVectorizer

In [136]:
vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=1)

In [137]:
analyze = vectorizer.build_analyzer()

In [138]:
import nltk

In [139]:
from collections import Counter

In [148]:
def  bigram_analysis(clusters,sentences):
    output = []
    for cluster in range(nclusters):
        print("cluster ",cluster,":")
        sentences_grouped = []
        frequencies = Counter([])
        for i,sentence in enumerate(clusters[cluster]):
            sentences_grouped.append(sentences[sentence])
            tokens = word_tokenize(sentences[sentence])
            bgs = nltk.bigrams(tokens)
            frequencies += Counter(bgs)
        output.append(nltk.FreqDist(frequencies))
    return output

       

In [149]:
bad_data = bigram_analysis(clusters_bad, sentences_bad)

cluster  0 :
cluster  1 :
cluster  2 :
cluster  3 :
cluster  4 :


In [158]:
good_data = bigram_analysis(clusters_good, sentences_good)

cluster  0 :
cluster  1 :
cluster  2 :
cluster  3 :
cluster  4 :


In [161]:
good_data[1].most_common(15)

[(('.', 'I'), 76),
 ((',', 'and'), 76),
 (('.', 'The'), 64),
 (('of', 'the'), 61),
 ((',', 'but'), 50),
 ((',', 'I'), 48),
 (('to', 'see'), 34),
 (('Titus', 'Welliver'), 30),
 (('is', 'a'), 28),
 ((',', 'the'), 27),
 (('to', 'the'), 26),
 (('the', 'books'), 24),
 (('and', 'I'), 23),
 (('I', "'m"), 22),
 (('.', 'It'), 22)]